Bibliotecas e módulos necessarios.

In [1]:
from bs4 import BeautifulSoup
import csv
import requests
from requests.exceptions import Timeout, RequestException, HTTPError

Nesse trecho de código estou fazendo a requisição e tratando possíveis erros.

In [2]:
URL = 'https://github.com/trending?spoken_language_code='
conteudo = None

try:
    resposta = requests.get(URL)
    resposta.raise_for_status()

except HTTPError as err_http:
    print(f'Erro de HTTP: {err_http}')

except ConnectionError as err_conn:
    print(f'Erro de conexão: {err_conn}')

except Timeout as err_timeout:
    print(f'Tempo de resposta excedido: {err_timeout}')

except RequestException as err_req:
    print(f'Erro na requisição: {err_req}')

except Exception as exc:
    print(f'Ocorreu um erro inesperado: {exc}')

else:
    conteudo = resposta.text


Criando um objeto **bs4** com o conteúdo da requisição e selecionando a **class** que contém as informações de interesse.

In [3]:
pagina = BeautifulSoup(conteudo, 'html.parser')
projetos = pagina.find_all('article', class_='Box-row')


Extraindo os dados, filtrando e limpando as informações de interesse, trasformando cada projeto e suas informações em uma lista e por fim adicionando a variável **conteudo_extraido**.

In [4]:
conteudo_extraido = list()
ranking = 1

for projeto in projetos[:10]:
    nome = projeto.h2.a.text.split()[-1]
    informacoes = projeto.find('div', class_='f6 color-fg-muted mt-2').text.split()[:6]
    del informacoes[3:5]
    informacoes.insert(0, nome)
    informacoes.insert(0, ranking)
    ranking += 1
    conteudo_extraido.append(informacoes)


Escrevendo o arquivo **CSV** com os **TOP 10** projetos do Github.

In [5]:
with open('github.csv', mode='w', encoding='utf-8') as arquivo:
    arquivo_csv = csv.writer(arquivo, delimiter=';')
    arquivo_csv.writerows([['ranking', 'project', 'language', 'stars', 'forks', 'stars_today']] + list(map(lambda row: row, conteudo_extraido)))